## Assignment 5-6: Document Classification
#### Summer 2021
**Authors:** GOAT Team (Esteban Aramayo, Ethan Haley, Claire Meyer, and Tyler Frankenburg)

In this assignment, we'll ingest a dataset on emails from the [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/Spambase) and build a classifier to predict if a row is a spam email or not, using the provided features.

In [1]:
import numpy as np
import pandas as pd 
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

##### Configuring the Spam dataset

First we'll import the email data into a CSV without headers, as we'll add the column names from the names file later.

In [2]:
spam_data = pd.read_csv("https://raw.githubusercontent.com/ebhtra/gory-graph/main/DocumentClassification/spambase.data", header=None)

spam_data.head()

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


Then we open the names file, which includes names for all our eventual features.

In [3]:
import urllib

link = "https://raw.githubusercontent.com/ebhtra/gory-graph/main/DocumentClassification/spambase.names"
f = urllib.request.urlopen(link)
f = f.read().decode()

Using Regex findall(), we can use pattern matching to ignore documentation text and pull in all the feature names to use as columns.

In [4]:
colnames = re.findall("word_freq_[a-z]*:|word_freq_[a-z]*[0-9]*[a-z]:|word_freq_[a-z]*[0-9]*:|char_freq_.:|capital_run_length_[a-z]*:",f)

In [5]:
len(colnames)

57

In [6]:
colnames.append("spam")

In [7]:
print(len(colnames))

58


In [8]:
spam_data.columns = colnames

In [9]:
spam_data.head()

,word_freq_make:,word_freq_address:,word_freq_all:,word_freq_3d:,word_freq_our:,word_freq_over:,word_freq_remove:,word_freq_internet:,word_freq_order:,word_freq_mail:,...,char_freq_;:,char_freq_(:,char_freq_[:,char_freq_!:,char_freq_$:,char_freq_#:,capital_run_length_average:,capital_run_length_longest:,capital_run_length_total:,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


##### Exploring the Data

Let's look at class balance, as well as descriptive statistics of each field.

In [10]:
spam_data.describe()

,word_freq_make:,word_freq_address:,word_freq_all:,word_freq_3d:,word_freq_our:,word_freq_over:,word_freq_remove:,word_freq_internet:,word_freq_order:,word_freq_mail:,...,char_freq_;:,char_freq_(:,char_freq_[:,char_freq_!:,char_freq_$:,char_freq_#:,capital_run_length_average:,capital_run_length_longest:,capital_run_length_total:,spam
count,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,...,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000,4601.000000
mean,0.104553,0.213015,0.280656,0.065425,0.312223,0.095901,0.114208,0.105295,0.090067,0.239413,...,0.038575,0.139030,0.016976,0.269071,0.075811,0.044238,5.191515,52.172789,283.289285,0.394045
std,0.305358,1.290575,0.504143,1.395151,0.672513,0.273824,0.391441,0.401071,0.278616,0.644755,...,0.243471,0.270355,0.109394,0.815672,0.245882,0.429342,31.729449,194.891310,606.347851,0.488698
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.588000,6.000000,35.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.065000,0.000000,0.000000,0.000000,0.000000,2.276000,15.000000,95.000000,0.000000
75%,0.000000,0.000000,0.420000,0.000000,0.380000,0.000000,0.000000,0.000000,0.000000,0.160000,...,0.000000,0.188000,0.000000,0.315000,0.052000,0.000000,3.706000,43.000000,266.000000,1.000000
max,4.540000,14.280000,5.100000,42.810000,10.000000,5.880000,7.270000,11.110000,5.260000,18.180000,...,4.385000,9.752000,4.081000,32.478000,6.003000,19.829000,1102.500000,9989.000000,15841.000000,1.000000


In [11]:
spam_data['spam'].value_counts()

0    2788
1    1813
Name: spam, dtype: int64

This dataset contains just under 40% spam, so the classes are not perfectly balanced.  

The capital letter counts are on a different scale from the string frequencies, so we'll need to normalize features for certain ML models.

##### Building the classifiers

We're going to squeeze what we can out of a few sci-kit learn classification models. To start, we'll split our target, y, from our features, X.

In [12]:
y = spam_data.iloc[:,57]
X = spam_data.iloc[:,:57]

Here are the features for the first email:

In [13]:
print(X.iloc[0,:])
print()
print(f"email is spam: {y[0]}")

word_freq_make:                  0.000
word_freq_address:               0.640
word_freq_all:                   0.640
word_freq_3d:                    0.000
word_freq_our:                   0.320
word_freq_over:                  0.000
word_freq_remove:                0.000
word_freq_internet:              0.000
word_freq_order:                 0.000
word_freq_mail:                  0.000
word_freq_receive:               0.000
word_freq_will:                  0.640
word_freq_people:                0.000
word_freq_report:                0.000
word_freq_addresses:             0.000
word_freq_free:                  0.320
word_freq_business:              0.000
word_freq_email:                 1.290
word_freq_you:                   1.930
word_freq_credit:                0.000
word_freq_your:                  0.960
word_freq_font:                  0.000
word_freq_000:                   0.000
word_freq_money:                 0.000
word_freq_hp:                    0.000
word_freq_hpl:           

There are frequency counts (how many times does this string appear in the email, for every 100 strings in the email) for 48 chosen strings, as well as 6 chosen characters.  There are also 3 different measures of capital letters.  Aside from trying to create our own non-linear combinations of the provided features, our feature engineering capabilities are handcuffed in this project, since we don't have the emails from which these features were chosen.  As such, we'll focus our efforts on leveraging sklearn's ML tools and best practices.

We'll split into test and train sets using sklearn's built-in splitting function. We'll set aside 10% of the samples as a holdout for testing.

In [14]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=.1, random_state=1234)

In [15]:
print('Resulting lengths of ')
print('X_train, y_train, X_test, and y_test:')
print(list(map(len, (X_train, y_train, X_test, y_test))))

Resulting lengths of 
X_train, y_train, X_test, and y_test:
[4140, 4140, 461, 461]


To allow classifiers like logistic regression to learn best, we'll normalize and shrink our features using sklearn's StandardScaler, which maps an array of numbers to a unit normal distribution by subtracting the mean of the array and dividing by its std.dev.

In [16]:
scaler = StandardScaler()
scaler.fit(X_train)  
X_scale_train = scaler.transform(X_train)
X_scale_test = scaler.transform(X_test)

In [17]:
print(f'Features now range from {np.min(X_scale_train)} to {np.max(X_scale_train)}')

Features now range from -0.9357153821855062 to 49.33457588472551


That maximum value 49 std.devs above the mean isn't ideal, but somehow one of the emails has over 15000 capitals. 

But in order to use k-fold cross-validation during the training of a model, it's better to fit the scaler to each training split and then transform the held-out fold with that current iteration of the scaler, thus avoiding data leakage within the process.  Rather than implementing it from scratch, we'll use a sklearn Pipeline object.

In [18]:
pipe = Pipeline([('scaler', StandardScaler()), ('model', LogisticRegression(random_state=1234, max_iter=500))])
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1234)
scores = cross_validate(pipe, X_train, y_train, scoring=('accuracy', 'precision'),
                        cv=cv, return_train_score=True, n_jobs=-1)

We began with Logistic Regression here because it's well suited to the type of features that were given, and allows us to get a good idea of which of those features are generally important or not.  We chose accuracy and precision as scoring metrics because of the nature of the classification task, spam filtering.  Besides just finding the most accurate filter in terms of correctly classifying emails, we want a filter with a high precision score, assuming we'd rather see a few spam emails in our inbox than to find a few important emails in the spam folder.

In [19]:
print(f"The mean accuracy during training 10 splits 3 different times was {round(np.mean(scores['train_accuracy']),4)}")
print(f"The standard deviation of those 30 scores was {round(np.std(scores['train_accuracy']),4)}")
print()
print(f"The mean scores for the 30 held out validation folds was {round(np.mean(scores['test_accuracy']),4)}")
print(f"Their st.dev. was {round(np.std(scores['test_accuracy']),4)}")

The mean accuracy during training 10 splits 3 different times was 0.9288
The standard deviation of those 30 scores was 0.002

The mean scores for the 30 held out validation folds was 0.9241
Their st.dev. was 0.0085


**The validation accuracies are just a half percent worse than the training ones, so there's not much evidence of overfitting.  With the 90-10 split provided by the 10 folds, we'd like to see the smaller (validation) folds showing $\sqrt{9}$ = 3 times higher stdev. than the training splits, which are 9 times larger.  Here we see that the ratio is actually 4.25 (8.5e-3 vs. 2e-3), so the 92.41% validation accuracy is a little unstable.** 

Let's see what the precisions were.

In [20]:
print(f"The mean test precision was {round(np.mean(scores['test_precision']),4)}")

The mean test precision was 0.9223


Since we used `RepeatedStratifiedKFold` for cross-validation, there should be about 40% spam in each fold, and since the model was about 93% accurate, it was probably predicting `spam` about 40% of the time.  With the precision score around 92%, this means we could expect around 8% False Positives on 40% spam predictions, or around 3% False Positives overall, where a non-spam email got sent to the spam box by mistake.  This may or may not be considered acceptable to a user of the filter, but we'll hope to do better with other models.

We might at this point choose to try out different regularization penalties (lasso and ridge regression) or to prune some of the 57 features we were given, but since there's little evidence of overfitting, we'll just retrain a logistic regression model on the entire training set and verify that it gets something like 92.5% accuracy with 3% false positives on the unseen 10% of the data we split off for testing.

In [25]:
# Use the scaled data from before
linear_reg = LogisticRegression(max_iter=300, random_state=620).fit(X_scale_train, y_train)
lin_pred = linear_reg.predict(X_scale_test)
lin_score = accuracy_score(y_test, lin_pred)
print(f"Test accuracy is {round(lin_score,4)}")
print(f"False positive rate is {round(sum(lin_pred - y_test == 1) / sum(y_test==0), 3)}")

Test accuracy is 0.9306
False positive rate is 0.05


In [26]:
print(f"Test precision is {round(np.dot(y_test, lin_pred) / sum(lin_pred),4)}")

Test precision is 0.9218


The test precision, accuracy, and therefore false positives, are as expected.  But it didn't necessarily have to turn out that way, with random variance in how any single data split happens to work, or how any one classifier happens to learn.  Let's just see how the 30 cross-validation models ranged:

In [24]:
print(f"Worst cross-val accuracy out of 30 splits was {round(min(scores['test_accuracy']),4)}")
print(f"Best was {round(max(scores['test_accuracy']),4)}")

Worst cross-val accuracy out of 30 splits was 0.9082
Best was 0.942


If we just chose which classifier to use for our spam filter based on how one test batch happened to perform, we might easily end up with the wrong choice.

**What were the 20 most important features?**

In [25]:
sorted(list(zip(linear_reg.coef_[0], X.columns)), key=lambda x: abs(x[0]), reverse=True)[:20]

[(-4.342655119799001, 'word_freq_george:'),
 (-2.6306171946409926, 'word_freq_hp:'),
 (-1.9651440530264461, 'word_freq_cs:'),
 (-1.483445991016186, 'word_freq_meeting:'),
 (1.3629780202839097, 'char_freq_$:'),
 (1.1582746904568435, 'capital_run_length_longest:'),
 (-1.1264679265650326, 'word_freq_lab:'),
 (-1.1225967091537659, 'word_freq_edu:'),
 (1.0141596061160116, 'word_freq_3d:'),
 (-0.9822463528948014, 'word_freq_hpl:'),
 (-0.9645475189839162, 'word_freq_85:'),
 (-0.9253869844251749, 'word_freq_project:'),
 (0.8852594401809076, 'word_freq_000:'),
 (0.862935252144216, 'word_freq_free:'),
 (0.8626591907835535, 'word_freq_remove:'),
 (-0.8439616518801258, 'word_freq_re:'),
 (-0.8257336316697627, 'word_freq_conference:'),
 (0.8116290737236534, 'char_freq_#:'),
 (0.5352714603142892, 'word_freq_credit:'),
 (-0.5339219619128391, 'word_freq_telnet:')]

**"$", long runs of capitals, "3d", "000", "free", and "remove" all suggest spam.**  

**"george", "hp", "cs", "meeting", "lab", and "edu" suggest non-spam.** 

Let's suppose these emails come from George who works at HP. It's debatable whether the frequency of the strings "George" and "HP" should be considered indications of non-spam.  If you want to train a separate classifier for every person then it makes sense to use their names.  But if you want to train a general purpose model, you wouldn't use names.  Still, this training set gives us no room to engineer other features, since we are just given a list of 57 of them, so perhaps it makes sense to use the name features.

Another topic of interest is how changing the model's threshold for classification changes the accuracy and precision.
Can we raise accuracy without letting too many important emails go to the spam folder (by lowering the threshold)?
Can we reduce false positives without hurting accuracy too much (by raising the threshold)?

In [44]:
def threshold(thresh, probs, truths):
    '''Use your own threshold as first argument, to see how
    accuracy and false positive rates change for given (2-D) probability
    array and ground truths as 2nd and 3rd arguments.
    '''
    assert len(truths) == probs.shape[0], print('X and y have different lengths')
    preds = [p[1] > thresh for p in probs]
    print(f"Threshold set to {thresh}")
    print(f"Test accuracy is {round(accuracy_score(truths, preds), 4)}")
    print(f"False positive rate is {round(sum(preds - truths == 1) / sum(truths==0), 3)}")

Now we'll ask the same model not for binary predictions of spam, but for probabilities of being spam, and then we can feed those probabilities into the threshold function we just made.

In [28]:
# default used by sklearn predict functions is 0.5, so this should match above scores
lin_prob = linear_reg.predict_proba(X_scale_test)
truths = y_test
threshold(0.5, lin_prob, truths)

Threshold set to 0.5
Test accuracy is 0.9306
False positive rate is 0.05


Raise threshold from sklearn's default 0.5

In [29]:
threshold(0.67, lin_prob, truths)

Threshold set to 0.67
Test accuracy is 0.9219
False positive rate is 0.025


**To reduce false positives by a half, we only had to lower accuracy from 0.931 to 0.922**

Lower threshold from 0.5

In [30]:
threshold(0.40, lin_prob, truths)

Threshold set to 0.4
Test accuracy is 0.9328
False positive rate is 0.076


**This direction looks worse:  Our FP rate went up 50% and we barely improved accuracy**

##### Decision Trees

Now we'll try some decision trees to see how they compare.  We won't use a pipeline for the cross-validation scaling here, since decision trees split on the order of the feature values, and the order won't be changed by scaling.  We will, however, use sklearn's `GridSearchCV` to find the best hyperparameters via cross-validation.  For decision trees, these hyperparameters include the depth of the tree and the minimum number of nodes needed in a leaf in order to split on it (amongst others). Both of those parameters are used to control the tradeoff between underfitting and overfitting.

In [31]:
grid_params_for_trees = [{
    'max_depth': [3,6,9,12],
    'min_samples_split': [2,3,4,5]}]

CV_trees = GridSearchCV(estimator = DecisionTreeClassifier(random_state=1234), scoring='accuracy',
                               param_grid = grid_params_for_trees,
                               cv = 10, return_train_score=True, n_jobs=-1)

CV_trees.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=1234),
             n_jobs=-1,
             param_grid=[{'max_depth': [3, 6, 9, 12],
                          'min_samples_split': [2, 3, 4, 5]}],
             return_train_score=True, scoring='accuracy')

In [32]:
# parameter combos with the highest validation scores
best_accuracy = pd.DataFrame(CV_trees.cv_results_).sort_values('mean_test_score', ascending=False)
best_accuracy.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
14,0.063571,0.003762,0.003685,0.000208,12,4,"{'max_depth': 12, 'min_samples_split': 4}",0.908213,0.915459,0.946860,...,0.977187,0.972356,0.974772,0.972088,0.975040,0.968867,0.973162,0.971820,0.973349,0.002239
10,0.050704,0.001575,0.003468,0.000191,9,4,"{'max_depth': 9, 'min_samples_split': 4}",0.905797,0.910628,0.934783,...,0.961621,0.957327,0.962158,0.958132,0.957059,0.954643,0.961084,0.958937,0.959071,0.002310
15,0.058885,0.005512,0.003046,0.000711,12,5,"{'max_depth': 12, 'min_samples_split': 5}",0.903382,0.903382,0.932367,...,0.975309,0.971014,0.973430,0.970209,0.974235,0.967525,0.971820,0.969404,0.971900,0.002389
12,0.061927,0.002012,0.003354,0.000123,12,2,"{'max_depth': 12, 'min_samples_split': 2}",0.903382,0.908213,0.946860,...,0.977456,0.974503,0.976382,0.973430,0.976919,0.970478,0.973698,0.974235,0.974960,0.002197
13,0.061335,0.002572,0.003652,0.000465,12,3,"{'max_depth': 12, 'min_samples_split': 3}",0.896135,0.908213,0.942029,...,0.977456,0.973698,0.975309,0.972893,0.976114,0.970478,0.973698,0.973430,0.974450,0.002156


In [33]:
best_accuracy[['mean_test_score', 'mean_train_score','param_max_depth','param_min_samples_split']].head()

,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split
14,0.924879,0.973349,12,4
10,0.922705,0.959071,9,4
15,0.921014,0.971900,12,5
12,0.921014,0.974960,12,2
13,0.921014,0.974450,12,3


The test scores are very similar to the LogisticRegression model results previously.  There is definitely some overfitting, since the training scores have about 30-50% the error rate of the validation folds.  This is because the best models have depths of 9 or even 12, which is deep enough to learn combinations of feature splits that don't generalize well.  On the other hand, the model is counteracting this tendency by using a minimum of 4 samples at a node in order to split on it, in the best models.

Let's train a model with the `max_depth=9` and `min_samples_split=4`, since it overfit less.  The results should be similar to what we just saw, but maybe the false positive rate is less than with logistic regression.

In [34]:
tree = DecisionTreeClassifier(max_depth=9, min_samples_split=4, random_state=1234).fit(X_train, y_train)
tree_probs = tree.predict_proba(X_test)

In [35]:
truths = y_test
threshold(0.5, tree_probs, truths)

Threshold set to 0.5
Test accuracy is 0.9349
False positive rate is 0.047


It seems like the tree got a little lucky on the 10% of emails we happened to hold out. Most importantly, it seems to have a very similar FP rate as the LogReg did.  We'll just try a couple of thresholds before moving on.

In [68]:
threshold(0.9, tree_probs, truths)

Threshold set to 0.9
Test accuracy is 0.9306
False positive rate is 0.029


**That's essentially as powerful as the LogReg classifier -- This tree with a high threshold has higher accuracy and slightly worse false positive rate than the LogReg with high threshold.**

For the tree, we used cross-validation to find the best parameters, and each combination of hyperparameters was used for 10 different splits, so we can feel somewhat confident that the results will generalize to new data (as long as it's all sampled from George's emails around this point in time).  But we only got accuracy scores, not precision, so let's run a few models to test the higher threshold and see how it affects false positives.

In [36]:
accs = []
fprs = []
thr = 0.9
for seed in range(10):
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=.1, random_state=seed)
    tree = DecisionTreeClassifier(max_depth=9, min_samples_split=4, random_state=seed).fit(X_tr, y_tr)
    tree_probs = tree.predict_proba(X_te)
    preds = [p[1] > thr for p in tree_probs]
    accs.append(accuracy_score(y_te, preds))
    fprs.append(sum(preds - y_te == 1) / sum(y_te==0))

In [37]:
print('Threshold was set to 0.9')
print(f"The mean accuracy over 10 different models was {round(np.mean(accs),4)}")
print(f"The mean FPR over 10 different models was {round(np.mean(fprs),4)}")

Threshold was set to 0.9
The mean accuracy over 10 different models was 0.9035
The mean FPR over 10 different models was 0.0415


**This indicates that the single model trained on just one split got a bit lucky, with 93% accuracy and 2.9% FPR.**

The UCI website that hosts this data says:
` False positives (marking good mail as spam) are very undesirable.  If we insist on zero false positives in the training/testing set, 20-25% of the spam passed through the filter.`
It's unclear what "zero false positives in the training/testing set" means here, but we'll try to build a classifier that consistently produces no FP's.  There are 40 spams per 100 emails.  We are searching for 0% FPR on the other 60, and aiming to let through less than 20% of those 40 spams, or less than 8% of all emails, meaning we need accuracy greater than 92%.

The tree variants that usually perform best on these tasks are `RandomForestClassifier` and `GradientBoostingClassifier`.

In [38]:
# As with all these GridSearchCV routines, training dozens of models can take awhile (~ 1 min here)
grid_params_for_forest = [{
    'max_depth': [6,9,12],
    'min_samples_split': [2,3,4]}]

CV_forest = GridSearchCV(estimator = RandomForestClassifier(n_estimators=500, random_state=1234), 
                         scoring='accuracy', param_grid = grid_params_for_forest,
                               cv = 10, return_train_score=True, n_jobs=-1)
# As with trees, we can use the unscaled data here
CV_forest.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=RandomForestClassifier(n_estimators=500,
                                              random_state=1234),
             n_jobs=-1,
             param_grid=[{'max_depth': [6, 9, 12],
                          'min_samples_split': [2, 3, 4]}],
             return_train_score=True, scoring='accuracy')

In [39]:
# parameter combos with the highest validation scores
best_accuracy = pd.DataFrame(CV_forest.cv_results_).sort_values('mean_test_score', ascending=False)
best_accuracy.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,...,split2_train_score,split3_train_score,split4_train_score,split5_train_score,split6_train_score,split7_train_score,split8_train_score,split9_train_score,mean_train_score,std_train_score
6,3.536149,0.037046,0.114863,0.005025,12,2,"{'max_depth': 12, 'min_samples_split': 2}",0.939614,0.934783,0.946860,...,0.979066,0.980140,0.979066,0.978798,0.979334,0.977724,0.978529,0.978798,0.979227,0.000833
7,3.575598,0.009654,0.110706,0.002506,12,3,"{'max_depth': 12, 'min_samples_split': 3}",0.934783,0.937198,0.946860,...,0.977187,0.977456,0.976382,0.976651,0.977187,0.976651,0.977992,0.977456,0.977268,0.000564
8,3.292135,0.557159,0.092053,0.022855,12,4,"{'max_depth': 12, 'min_samples_split': 4}",0.934783,0.937198,0.946860,...,0.976919,0.976651,0.975040,0.973430,0.976919,0.974772,0.974772,0.975845,0.975926,0.001321
3,3.118102,0.048349,0.102361,0.001990,9,2,"{'max_depth': 9, 'min_samples_split': 2}",0.934783,0.929952,0.937198,...,0.961621,0.961889,0.961084,0.961889,0.962426,0.960816,0.962158,0.961889,0.961970,0.000679
4,3.192847,0.018279,0.103052,0.003374,9,3,"{'max_depth': 9, 'min_samples_split': 3}",0.937198,0.929952,0.937198,...,0.961084,0.960548,0.960816,0.960548,0.960816,0.961084,0.962158,0.961621,0.961218,0.000615


In [40]:
best_accuracy[['mean_test_score', 'mean_train_score','param_max_depth','param_min_samples_split']].head()

,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split
6,0.946135,0.979227,12,2
7,0.945169,0.977268,12,3
8,0.944203,0.975926,12,4
3,0.939855,0.961970,9,2
4,0.939614,0.961218,9,3


Those are even better scores, but the CV search chose the most overfittable params -- highest depth and smallest split.  Since random forests are somewhat resistant to overfitting, we might try even more extreme numbers.

In [41]:
grid_params_for_forest = [{
    'max_depth': [12,14,16],
    'min_samples_split': [2,3]}]

CV_forest = GridSearchCV(estimator = RandomForestClassifier(n_estimators=500, random_state=1234), scoring='accuracy',
                               param_grid = grid_params_for_forest,
                               cv = 10, return_train_score=True, n_jobs=-1)

CV_forest.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=RandomForestClassifier(n_estimators=500,
                                              random_state=1234),
             n_jobs=-1,
             param_grid=[{'max_depth': [12, 14, 16],
                          'min_samples_split': [2, 3]}],
             return_train_score=True, scoring='accuracy')

In [42]:
# parameter combos with the highest validation scores
best_accuracy = pd.DataFrame(CV_forest.cv_results_).sort_values('mean_test_score', ascending=False)
best_accuracy[['mean_test_score', 'mean_train_score','param_max_depth','param_min_samples_split']].head()

,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split
5,0.950725,0.989962,16,3
4,0.950725,0.990821,16,2
2,0.948551,0.987493,14,2
3,0.948068,0.986098,14,3
0,0.946135,0.979227,12,2


In [45]:
# Use the best params from above
forest = RandomForestClassifier(max_depth=16, min_samples_split=3,
                                n_estimators=500, random_state=1234).fit(X_train, y_train)
forest_probs = forest.predict_proba(X_test)
threshold(0.5, forest_probs, y_test)

Threshold set to 0.5
Test accuracy is 0.9566
False positive rate is 0.025


Raise the threshold to where no good mail gets filtered

In [46]:
threshold(0.9, forest_probs, y_test)

Threshold set to 0.9
Test accuracy is 0.8568
False positive rate is 0.0


**This random forest model has an accuracy over 95% if you're OK with a 2.5% FPR, but drops to 85.7% if you need 0% FPR**

Let's get ten more opinions before moving on to gradient boosting.

In [47]:
accs = []
fprs = []
for seed in range(10):
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=.1, random_state=seed)
    forest = RandomForestClassifier(max_depth=16, min_samples_split=3,
                                n_estimators=500, random_state=seed).fit(X_tr, y_tr)
    forest_probs = forest.predict_proba(X_te)
    accs.append(accuracy_score(y_te, [p[1] > 0.9 for p in forest_probs]))
    fprs.append(sum([p[1] > 0.9 for p in forest_probs] - y_te == 1) / sum(y_te==0))

In [48]:
print('Threshold was set to 0.9')
print(f"The mean accuracy over 10 different models was {round(np.mean(accs),4)}")
print(f"The mean FPR over 10 different models was {round(np.mean(fprs),4)}")

Threshold was set to 0.9
The mean accuracy over 10 different models was 0.8486
The mean FPR over 10 different models was 0.0022


**That won't cut it. Also it sent a couple of good emails to spam.**

Let's try gradient boosted trees.

In [53]:
# Training time and overfitting are chiefly managed by n_iter_no_change parameter
grid_params_for_boosting = [{
    'max_depth': [3,4],
    'min_samples_split': [4,5,6],
    'learning_rate':[0.15, 0.3]}]

CV_boost = GridSearchCV(estimator = GradientBoostingClassifier(n_estimators=500, random_state=1234, n_iter_no_change=4),
                        scoring='accuracy', param_grid = grid_params_for_boosting,
                        cv = 10, return_train_score=True, n_jobs=-1)
# use whole dataset this time, since we'll rebuild a model anyways
CV_boost.fit(X, y)

GridSearchCV(cv=10,
             estimator=GradientBoostingClassifier(n_estimators=500,
                                                  n_iter_no_change=4,
                                                  random_state=1234),
             n_jobs=-1,
             param_grid=[{'learning_rate': [0.15, 0.3], 'max_depth': [3, 4],
                          'min_samples_split': [4, 5, 6]}],
             return_train_score=True, scoring='accuracy')

In [54]:
# parameter combos with the highest validation scores
best_accuracy = pd.DataFrame(CV_boost.cv_results_).sort_values('mean_test_score', ascending=False)
best_accuracy[['mean_test_score', 'mean_train_score','param_max_depth','param_min_samples_split','param_learning_rate']].head()

,mean_test_score,mean_train_score,param_max_depth,param_min_samples_split,param_learning_rate
1,0.941531,0.966384,3,5,0.15
5,0.940662,0.973339,4,6,0.15
0,0.940661,0.967688,3,4,0.15
8,0.940009,0.968123,3,6,0.3
4,0.939793,0.973581,4,5,0.15


Let's see what the FP rate is.

In [57]:
# Use the shallowest depth and the higher min_samples_split
booster = GradientBoostingClassifier(max_depth=3, min_samples_split=5, learning_rate=0.15,
                                n_estimators=500, random_state=1234).fit(X_train, y_train)
booster_probs = booster.predict_proba(X_test)
threshold(0.5, booster_probs, y_test)

Threshold set to 0.5
Test accuracy is 0.9588
False positive rate is 0.036


And with a higher classification threshold?

In [58]:
threshold(0.9, booster_probs, y_test)

Threshold set to 0.9
Test accuracy is 0.9414
False positive rate is 0.014


**That's the most powerful classifier yet, but is still not getting 0% FPR.**  

Let's see once more how it generalizes to 10 random fits and splits.

In [63]:
accs = []
fprs = []
thr = 0.9
for rs in range(10):
    X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=.1, random_state=rs)
    booster = GradientBoostingClassifier(max_depth=3, min_samples_split=5, learning_rate=0.15,
                                n_estimators=500, random_state=rs).fit(X_tr, y_tr)
    booster_probs = booster.predict_proba(X_te)
    preds = [p[1] > thr for p in booster_probs]
    accs.append(accuracy_score(y_te, preds))
    fprs.append(sum(preds - y_te == 1) / sum(y_te==0))

In [65]:
print(f'Threshold was set to {thr}')
print(f"The mean accuracy over 10 different models was {round(np.mean(accs),4)}")
print(f"The mean FPR over 10 different models was {round(np.mean(fprs),4)}")

Threshold was set to 0.9
The mean accuracy over 10 different models was 0.9388
The mean FPR over 10 different models was 0.0112


It's hard to know how to interpret these results.  We've used some moderately sophisticated algorithms and techniques, and yet we can't eliminate false positives over many repeated runs.  It makes you (us) wonder what the hosts of this dataset mean by "insisting on zero false positives". How is that even possible?  Did they run one single split and fit and get such a result?  Did they then try it on every possible split and fit? If we gave them George's next 1000 emails, how much would they bet on zero false positives?

**Were the same features important for the GBC as for LogReg?**

In [67]:
print('GBC most important')
sorted(list(zip(booster.feature_importances_, X.columns)), reverse=True)[:10]

GBC most important


[(0.2325409507096751, 'char_freq_!:'),
 (0.19307013396261702, 'char_freq_$:'),
 (0.1174214199362354, 'word_freq_remove:'),
 (0.08773718427150339, 'word_freq_hp:'),
 (0.05811219648456616, 'word_freq_free:'),
 (0.057012463917724715, 'capital_run_length_average:'),
 (0.03435625166201122, 'word_freq_your:'),
 (0.03177906060287507, 'capital_run_length_longest:'),
 (0.028981346156235873, 'word_freq_george:'),
 (0.026140152403960816, 'word_freq_edu:')]

In [68]:
print('LogReg most important')
sorted(list(zip(linear_reg.coef_[0], X.columns)), key=lambda x: abs(x[0]), reverse=True)[:10]

LogReg most important


[(-4.342655119799001, 'word_freq_george:'),
 (-2.6306171946409926, 'word_freq_hp:'),
 (-1.9651440530264461, 'word_freq_cs:'),
 (-1.483445991016186, 'word_freq_meeting:'),
 (1.3629780202839097, 'char_freq_$:'),
 (1.1582746904568435, 'capital_run_length_longest:'),
 (-1.1264679265650326, 'word_freq_lab:'),
 (-1.1225967091537659, 'word_freq_edu:'),
 (1.0141596061160116, 'word_freq_3d:'),
 (-0.9822463528948014, 'word_freq_hpl:')]

There's a lot of overlap, but the Gradient Boosted model leaned heavily on exclamation points, presumably to indicate spam, whereas the Linear model banked more on what indicated good emails, like the user-specific names.

### Summary

Constrained to a pre-fixed set of features and goals, we turned our focus to using some of the most common and powerful ML tools.  
- Training Classifiers to generalize to unseen inputs
 - Scaling features appropriately
 - Splitting data into subsets where necessary
 - Using pipelines with cross-validation
 - Using grid search with cross-validation to optimize hyperparameters  
 
 
- Evaluating the trained Classifiers
 - Choosing appropriate metrics 
 - Interpreting repeatability/generalizability of test results
 - Avoiding test data leakage
 - Raising classification thresholds to eliminate false positives
 - Interpreting learned features

Our most powerful model was the gradient boosted classifier with a high, pre-set threshold, which was 93.9% accurate on multiple data splits.  Unfortunately it still sent 1.1% of good emails to the spam folder.